In [1]:
import matplotlib.pyplot as plt
import plotly.express as px
import pandas_profiling
import pandas as pd
import os

## Simple Data Processing

In [4]:
PATH = os.path.join("./data_raw", "resale_flat_prices")

In [3]:
files = os.listdir(PATH)
data_files = [fname for fname in files if ".csv" in fname and "resale-flat-prices" in fname]
df_list = []

for fname in data_files:
  df = pd.read_csv(os.path.join(PATH, fname))
  df_list.append(df)

df_combined = pd.concat(df_list)

In [4]:
df_combined.sort_values(by="month", ascending=True, inplace=True)
df_combined.reset_index(drop=True, inplace=True)
df_combined["flat_model"] = df_combined["flat_model"].str.upper()
df_combined.tail(10)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
880838,2022-09,GEYLANG,3 ROOM,91,PAYA LEBAR WAY,10 TO 12,67.0,IMPROVED,1972,328000.0,49 years 01 month
880839,2022-09,GEYLANG,3 ROOM,60,CIRCUIT RD,04 TO 06,56.0,STANDARD,1969,320000.0,45 years 05 months
880840,2022-09,CHOA CHU KANG,EXECUTIVE,134,TECK WHYE LANE,10 TO 12,144.0,APARTMENT,1993,738000.0,69 years 10 months
880841,2022-09,CHOA CHU KANG,5 ROOM,453,CHOA CHU KANG AVE 4,07 TO 09,123.0,PREMIUM APARTMENT,2000,525000.0,76 years 06 months
880842,2022-09,CHOA CHU KANG,4 ROOM,5,TECK WHYE AVE,04 TO 06,104.0,MODEL A,1984,451000.0,60 years 08 months
880843,2022-09,CHOA CHU KANG,4 ROOM,816B,KEAT HONG LINK,13 TO 15,92.0,MODEL A,2017,565000.0,94 years 01 month
880844,2022-09,CHOA CHU KANG,4 ROOM,691A,CHOA CHU KANG CRES,16 TO 18,91.0,MODEL A,2003,465000.0,79 years 09 months
880845,2022-09,CHOA CHU KANG,4 ROOM,684D,CHOA CHU KANG CRES,13 TO 15,90.0,MODEL A,2002,460000.0,79 years 01 month
880846,2022-09,CHOA CHU KANG,4 ROOM,487C,CHOA CHU KANG AVE 5,13 TO 15,93.0,MODEL A,2016,520000.0,92 years 10 months
880847,2022-09,YISHUN,5 ROOM,677C,YISHUN RING RD,04 TO 06,113.0,IMPROVED,2018,655000.0,95 years 02 months


In [5]:
df_combined.to_csv(os.path.join(PATH, "resale_flat_prices_all.csv"), index=False)

## Exploratory Data Analysis

We first use `.describe()` and `.corr()` on `df_combined` to do some very preliminary analysis.

In [9]:
df_combined.describe()

,floor_area_sqm,lease_commence_date,resale_price
count,880848.000000,880848.000000,8.808480e+05
mean,95.706267,1987.764496,3.073677e+05
std,25.931028,10.124900,1.592593e+05
min,28.000000,1966.000000,5.000000e+03
25%,73.000000,1980.000000,1.880000e+05
50%,93.000000,1986.000000,2.860000e+05
75%,113.000000,1995.000000,4.000000e+05
max,307.000000,2019.000000,1.418000e+06


In [10]:
df_combined.corr()

,floor_area_sqm,lease_commence_date,resale_price
floor_area_sqm,1.000000,0.422305,0.595527
lease_commence_date,0.422305,1.000000,0.522073
resale_price,0.595527,0.522073,1.000000


In this section, we will plot histograms categorical feature values to determine which of them have smaller counts. We will then categorise said features that have small counts as `other`.
The yellowish-orange lines signify the boundaries that we have drawn to do the above preprocessing step.

In [16]:
df_combined["flat_model"].value_counts().plot(kind="barh", grid=True, title="Counts of different models of flats",
                                              ylabel="Flat Model", figsize=(10, 6))
plt.axvline(9000, alpha=0.5, c="orange", label="Count = 9000")
plt.legend()
plt.savefig(os.path.join(PATH, "flat_model.png"), facecolor="white", pad_inches=0.3, bbox_inches="tight")
plt.clf()

In [17]:
df_combined["town"].value_counts().plot(kind="barh", grid=True, title="Counts of different towns",
                                              ylabel="Towns", figsize=(8, 10))
plt.axvline(12000, alpha=0.5, c="orange", label="Count = 12000")
plt.legend()
plt.savefig(os.path.join(PATH, "town.png"), facecolor="white", pad_inches=0.3, bbox_inches="tight")
plt.clf()

In [18]:
df_combined["flat_type"].value_counts().plot(kind="barh", grid=True, title="Counts of flat types",
                                              ylabel="Flat Type", figsize=(8, 5))
plt.axvline(50000, alpha=0.5, c="orange", label="Count = 50000")
plt.legend()
plt.savefig(os.path.join(PATH, "flat_type.png"), facecolor="white", pad_inches=0.3, bbox_inches="tight")
plt.clf()

In the next step, we will use `pandas_profiling` to automate our standard exploratory data analysis processes.

In [14]:
profile = pandas_profiling.ProfileReport(df_combined, title="Report on df_combined")

In [15]:
profile.to_file(os.path.join(PATH, "df_combined_report.html"))

Summarize dataset:  64%|██████▎   | 14/22 [00:04<00:01,  6.69it/s, Calculate kendall correlation]       /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4868: RuntimeWarning: overflow encountered in long_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))
Export report to file: 100%|██████████| 1/1 [00:00<00:00, 569.49it/s]


In [14]:
PROCESSED_PATH = os.path.join("./data_processed", "resale_flat_prices")
df_new = pd.read_csv(os.path.join(PROCESSED_PATH, "engineered_data.csv"))
new_profile = pandas_profiling.ProfileReport(df_new, title="Report on df_new")
new_profile.to_file(os.path.join(PROCESSED_PATH, "df_new_report.pdf"))

Summarize dataset:  78%|███████▊  | 28/36 [00:07<00:02,  2.84it/s, Calculate kendall correlation]            /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4868: RuntimeWarning: overflow encountered in long_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))
Render HTML: 100%|██████████| 1/1 [00:04<00:00,  4.30s/it]
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas_profiling/profile_report.py:262: UserWarning: Extension .pdf not supported. For now we assume .html was intended. To remove this warning, please use .html or .json.
  warnings.warn(
Export report to file: 100%|██████████| 1/1 [00:00<00:00, 70.85it/s]
